# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> OK </b>
- Nearest Neighbors OK
    

# Leitura dos arquivos

In [2]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features



In [3]:
market = pd.read_csv('../../estaticos_market.csv')

In [38]:
port1 = pd.read_csv('../../estaticos_portfolio1.csv')

In [39]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [40]:
port1.shape

(555, 182)

In [6]:
market.drop(columns=exclude_variables, inplace= True)
market.drop(columns=['Unnamed: 0'], inplace= True)
market.set_index('id',inplace=True)
market.shape

(462298, 37)

In [41]:
port1.drop(columns=exclude_variables, inplace= True)
port1.drop(columns=['Unnamed: 0'], inplace= True)
port1.set_index('id',inplace=True)
port1.shape

(555, 37)

In [8]:
#p1.to_csv('estaticos_49_features.csv')

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [9]:
fill_dict = market.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict

{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [10]:
market.fillna(value=fill_dict, inplace=True)
market[cat2bool] = market[cat2bool].astype(bool)

bool2numeric = market.columns[market.dtypes == bool]
market[bool2numeric] = market[bool2numeric].astype(int)

In [11]:
#market_p1.head(2)

## Encoding

In [12]:
cat = market.columns[market.dtypes == object]
not_cat = market.columns[market.dtypes != object]

In [13]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [14]:
market_le = pd.DataFrame()
market_le[cat] = market[cat].apply(lambda col: le.fit_transform(col))
market_encoded = pd.concat([market[not_cat],market_le], axis =1)
market_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

Aqui fiz fiz um novo dataset para os ids do portifolio1 e o merge com os dados já aplicados o LabelEncoder

In [42]:
port1_le = pd.DataFrame(index=port1.index)
port1_le = market_encoded.merge(port1_le, how='right',left_index=True,right_index=True)


In [43]:
port1_le.head(3)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9,1,0.646575,0,0,1,0,0,0,0,0,...,30,9,16,2,4,1,14,61,10,10
32e5f4e10932153a7ba869cb0386e7e02d49d2461046b866411556d2af150702,0,6.586301,0,0,0,0,0,1,1,0,...,30,9,9,4,0,2,3,44,10,7
a95d6f30bba445bd3d6b0c5b36f865b38ec01d1733609095d359649cfedf15d1,1,8.010959,0,0,1,0,0,0,0,1,...,30,9,9,1,0,2,8,0,1,1


# Nearest Neighbors em uma amostra do banco

Como não temos o que prever, usamos o treinamento não supervisionado do Nearest Neighbors



Apenas amostra por enquanto

Precisa escolher o numero de componentes a indicar

In [17]:
market_sample = market_encoded#.sample(n=50000)

In [18]:
%%time
from sklearn.neighbors import NearestNeighbors

qtd_neighbors = 5
nearest = NearestNeighbors(n_neighbors=qtd_neighbors, metric = 'cosine')

nearest.fit(market_sample)


CPU times: user 194 ms, sys: 177 ms, total: 371 ms
Wall time: 401 ms


NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

Dict para a lista de vizinhos mais próximos

In [44]:
%%time
neighbors_list = {}

for row in range(port1_le.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(port1_le.iloc[[row]].values)

CPU times: user 4min 16s, sys: 2min 29s, total: 6min 45s
Wall time: 3min 5s


A resposta é uma lista da distância e uma lista de componentes. A componente é a linha do Dataset, ou seja, clientes.

In [46]:
neighbors_list

{0: (array([[0.00000000e+00, 1.98039241e-09, 2.97804581e-09, 3.25201976e-09,
          4.30962777e-09]]),
  array([[   369, 283497,  86123,  14002, 376227]])),
 1: (array([[0.00000000e+00, 3.24501609e-07, 3.45030461e-07, 4.02407573e-07,
          5.11891993e-07]]),
  array([[   604, 448287, 327525, 368594, 312928]])),
 2: (array([[0.00000000e+00, 4.21165403e-09, 1.46899684e-08, 2.04650112e-08,
          4.13484105e-08]]),
  array([[  1703, 293620, 250766, 324406, 219108]])),
 3: (array([[5.55111512e-16, 8.08250117e-08, 8.63704627e-08, 9.21093004e-08,
          9.34973678e-08]]),
  array([[  2521, 106875, 196371,  26381, 419356]])),
 4: (array([[0.00000000e+00, 6.01917227e-09, 9.43644634e-08, 1.11633338e-07,
          1.59318892e-07]]),
  array([[  5788, 323055, 441665, 128139,  55536]])),
 5: (array([[0.00000000e+00, 3.90187294e-09, 4.21316182e-09, 4.70753314e-09,
          4.93831498e-09]]),
  array([[  6369, 269763, 398847, 247222, 177778]])),
 6: (array([[1.11022302e-16, 1.70028436e

In [45]:
len(neighbors_list)

555

## Avaliação

Agora com o NN funcionando, precisa confrontar as sugestões para saber qual é o mais sugerido.

## Verificando o que foi mais sugerido

Da lista de vizinhos, listar em ordem os mais frequentes.

In [47]:
list_size = len(neighbors_list)
num_neighbors = len(neighbors_list[0][1][0])

neighbors_idx_array = neighbors_list[0][1][0]
neighbors_distance_array = neighbors_list[0][0][0]
np.delete(neighbors_idx_array, [0,1])
np.delete(neighbors_distance_array, [0,1])
for line in range(1,list_size):
    neighbors_idx_array = np.concatenate((neighbors_idx_array, neighbors_list[line][1][0]), axis=None) 
    neighbors_distance_array = np.concatenate((neighbors_distance_array, neighbors_list[line][0][0]), axis=None) 

if len(neighbors_idx_array) != list_size*num_neighbors:
    print("ERROR: Check array size.")
# Temos agora um array unidimensional com os índices dos clientes recomendados

np.delete(neighbors_idx_array, [0,1])
np.delete(neighbors_distance_array, [0,1])

array([2.97804581e-09, 3.25201976e-09, 4.30962777e-09, ...,
       4.30537725e-07, 6.91101884e-07, 1.64109704e-06])

In [49]:
neighbors_distance_array[0]

0.0

Criar um dicionário com o número da linha, id e distância

In [50]:
dicio = {}
for idx,ind in zip(neighbors_idx_array, range(len(neighbors_idx_array))):
    dicio[ind] = (port1.iloc[int(ind/qtd_neighbors)].name, market_sample.iloc[idx].name, (neighbors_distance_array[ind]))

In [51]:
dicio

{0: ('dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9',
  'dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9',
  0.0),
 1: ('dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9',
  'db74ecba50a1b89b9d65304d8bf04af634c5c14e29f2884238a097ee4fb888ac',
  1.9803924056205346e-09),
 2: ('dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9',
  'fba00012442dcd8a7ac32365051c25c161a1d58497da6e5d68d184dd8849c51f',
  2.9780458099537555e-09),
 3: ('dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9',
  'fa8e392e41c24aa03bb478288554458e581ea0a8a18293d1db2c7007ac7d8def',
  3.252019764587999e-09),
 4: ('dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9',
  '3a3643d77d7150dbaefb7b35e37a70d0d8128013365b4ea598a4d224ef8ed260',
  4.309627765763935e-09),
 5: ('32e5f4e10932153a7ba869cb0386e7e02d49d2461046b866411556d2af150702',
  '32e5f4e10932153a7ba869cb0386e7e02d49d2461046b866411556d2af150702',
  0.0),
 6: ('32e5f4e1093215

In [52]:
len(dicio)

2775

In [25]:
neig_df = pd.DataFrame.from_dict(dicio,orient='index')
neig_df.rename(columns={0:'id_origin', 1:'id',2:'distance'},inplace = True)
neig_df.set_index('id', inplace=True)

In [26]:
conflicts = neig_df.merge(port1, how='inner',left_index=True,right_index=True)
suggestion_with_conflicts = neig_df.merge(market, how='left',left_index=True,right_index=True)
#neig_df.shape, port1.shape, conflicts.shape, suggestion_with_conflicts.shape, 
suggestions = suggestion_with_conflicts.drop(conflicts.index)

In [27]:
len(conflicts.index.unique()), conflicts.shape, suggestion_with_conflicts.shape, port1.shape, suggestions.shape

(566, (1202, 39), (2830, 39), (566, 37), (1628, 39))

In [28]:
suggestions.head(5)

,id_origin,distance,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,idade_media_socios,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux
id,,,,,,,,,,,,,,,,,,,,,
0012f06c4ea31b52d25c6d791ec7f6c087bd7c0eb1f338c2400b987ee6e2c22b,d8e21466fc2ab4b21c3456be1594c93abb67fc49127a8d...,5.202584e-09,1,MUNICIPIO,RO,OUTROS,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,31.150685,> 20,...,other,other,1,0.0,0.0,0.0,"DE R$ 100.000.000,01 A R$ 300.000.000,00","DE R$ 100.000.000,01 A R$ 300.000.000,00",104474450.0,105614450.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,6940a00c907814c5252c927e5f9d6e6f4ba29612bcd90a...,8.879453e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.0,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
00489ba3f0d05b82ff626d66bf32769dff5ba75303bc9b48f2e971e14369de79,5b0e32f5b3c58c14ccbba2da37b1b718ba25bee3bf1e30...,5.541567e-12,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,SERVICOS DE EDUCACAO,SERVIÇO,17.312329,15 a 20,...,OESTE MARANHENSE,IMPERATRIZ,1,3.0,58.0,1.0,"DE R$ 30.000.000,01 A R$ 100.000.000,00","DE R$ 30.000.000,01 A R$ 100.000.000,00",58058532.0,58058532.0
0086ad83094c5903c631bb3028c022ec7f15391f7bbb7ba2d5036588db7547a1,7fc36495804caf524926b223b90e2e90eb713db15d1059...,2.450540e-11,1,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,SERVICOS ADMINISTRATIVOS,SERVIÇO,12.032877,10 a 15,...,CENTRO AMAZONENSE,MANAUS,1,1.0,67.0,0.0,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",12363840.0,12363840.0
00ac2b0f09807d4304312374622e5e6a99337ba14858d30f9f546dc01fc8c5de,5c7e219e68b5521c3268c287ee25e945d36e8fbfd0f2b6...,2.052691e-11,1,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,PRODUTOS DE AGROPECUARIA,AGROPECUARIA,22.353425,> 20,...,LESTE MARANHENSE,CODO,0,3.0,66.0,1.0,"DE R$ 10.000.000,01 A R$ 30.000.000,00","DE R$ 10.000.000,01 A R$ 30.000.000,00",16691184.0,16691184.0


# Visualização dos resultados

## Mapa (Heat)

### Carrega arquivo com informações geográficas
Gerado por `analysis/Save_IBGE_data_for_Map_Visualization_in_Recommendation_System.ipynb`

In [29]:
coord_micro = pd.read_csv('../recommender_system_squad_1/coord_geograficas_microrregioes_brasil.csv')

In [30]:
### Importa módulo Folium
import folium
from folium import plugins
import random

### Gera lista com coordenadas das sugestões (suggestions)

In [31]:
map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in suggestions['nm_micro_regiao']:
    coord = coord_micro[coord_micro['nm_micro'] == local][['lat', 'lng']]
    if not coord.empty:
        map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.25),
                          coord['lng'].values[0]-random.uniform(0, 0.25)])
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning
print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 225


### Mostra heat map

In [32]:
mapa = folium.Map(location=[-8.788497,-53.879873],tiles='Stamen Toner',zoom_start=5)

mapa.add_child(plugins.HeatMap(map_coord))
mapa

## Mapa com marcadores
Passe o mouse sobre os marcadores para informação adicional.

In [33]:
# Para não ter repetições no mapa com marcadores, vou criar um novo df a partir de suggestions,
# deixando só uma entrada para cada id do market
unique_suggestions = pd.DataFrame()
for client in suggestions['id_origin'].unique():
    idxmin_distance = suggestions[suggestions['id_origin']==client]['distance'].idxmin()
    unique_suggestions = unique_suggestions.append(market.loc[idxmin_distance])

In [34]:
uniq_map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in zip(unique_suggestions[['nm_micro_regiao', 'de_ramo']].to_numpy(),
                 unique_suggestions[['nm_micro_regiao', 'de_ramo']].index):
    coord = coord_micro[coord_micro['nm_micro'] == local[0][0]][['lat', 'lng']]
    if not coord.empty:
        uniq_map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.5),
                          coord['lng'].values[0]-random.uniform(0, 0.5),
                          'id:'+local[1][:10]+' ramo:'+local[0][1]]) # para o popup com 
                          #informação do id e ramo da empresa de sugestão.
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning

print('Suggestions without info about nm_micro:', null_count)

Suggestions without info about nm_micro: 84


In [35]:
#incluir também clientes atuais
current_map_coord = []
null_count = 0  # para verificar quantas sugestões não tem informação da microrregião
for local in zip(port1[['nm_micro_regiao', 'de_ramo']].to_numpy(),
                 port1[['nm_micro_regiao', 'de_ramo']].index):
    coord = coord_micro[coord_micro['nm_micro'] == local[0][0]][['lat', 'lng']]
    if not coord.empty:
        current_map_coord.append([coord['lat'].values[0]-random.uniform(0, 0.5),
                          coord['lng'].values[0]-random.uniform(0, 0.5),
                          'id:'+str(local[1])[:10]+' ramo:'+local[0][1]]) # para o popup com 
                          #informação do id e ramo da empresa de sugestão.
    else:
        null_count = null_count + 1
        # print('Location null', null_count)   #TODO: mostrar como warning

print('Current clients without info about nm_micro:', null_count)

Current clients without info about nm_micro: 70


In [37]:
# Mostra marcadores com informação do id e ramo da empresa de sugestão.
markmapa = folium.Map(location=[-8.788497, -53.879873],
                  tiles='OpenStreetMap', zoom_start=5)

# tooltip = 'Click for info'
#clientes atuais
# for mark in current_map_coord:
#     folium.Marker([mark[0], mark[1]],
#                  popup = mark[2],
#                  tooltip = mark[2],
#                  icon=folium.Icon(icon='ok', color='green')).add_to(markmapa)
    
for mark in uniq_map_coord:
    # sugestões
    folium.Marker([mark[0], mark[1]], popup=mark[2], tooltip=mark[2]).add_to(markmapa)

#markmapa.add_child(plugins.HeatMap(map_coord))
markmapa


## Lista ordenada por faixa de faturamento